In [2]:
import os
import time

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, merge

/home/admin_user/anaconda3/envs/gsshop/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/admin_user/anaconda3/envs/gsshop/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
COLUMNS = ["age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
          "occupation", "relationship", "race", "gender", "capital_gain", "capital_loss", "hours_per_week", "native_country", "income_bracket"]
LABEL_COLUMN = "label"
CATEGORICAL_COLUMNS = ["workclass", "education", "marital_status", "occupation", "relationship", "race", "gender", "native_country"]
NUMERICAL_COLUMNS = ["age", "education_num", "capital_gain", "capital_loss", "hours_per_week"]

In [4]:
df_train = pd.read_csv('adult.data.csv', header=None, names=COLUMNS)
df_test = pd.read_csv('adult.test.csv', header=None, names=COLUMNS)

df_train[LABEL_COLUMN] = df_train['income_bracket'].apply(lambda x: ">50K" in x).astype(int)
df_test[LABEL_COLUMN] = df_test['income_bracket'].apply(lambda x: ">50K" in x).astype(int)

y_train = df_train[LABEL_COLUMN].values
y_test = df_test[LABEL_COLUMN].values

df_train.pop(LABEL_COLUMN)
df_test.pop(LABEL_COLUMN)

x_train = df_train.values
x_test = df_test.values

In [27]:
class Wide: